# TP3


# Neural Networks



Les réseaux de neurones peuvent être construits à l'aide du package ``torch.nn``.

Maintenant que vous avez eu un aperçu de « autograd », « nn » dépend de
``autograd`` pour définir des modèles et les différencier.
Un ``nn.Module`` contient des couches et une méthode ``forward(input)`` qui
renvoie la « sortie ».

Une procédure d'entraînement typique pour un réseau de neurones est la suivante :

- Définir le réseau neuronal qui possède certains paramètres à apprendre (ou
  poids)
- Itérer sur un ensemble de données d'entrées
- Calculer la perte (dans quelle mesure la sortie est-elle correcte)
- Faire de la backpropagation dans les paramètres du réseau
- Mettre à jour les poids du réseau, généralement à l'aide d'une simple règle de mise à jour :
  ``poids = poids - taux d'apprentissage * gradient``

## Définir le réseau

Définissons un réseau :

In [1]:
%matplotlib inline

In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


 <font color='blue'> **Question** : Décrire l'architecture du réseau </font>

 <font color='blue'> **Question** : Afficher les poids et biais de la première convolution </font>

In [4]:
print(net.conv1.weight.shape)
print(net.conv1.bias.shape)

print(net.conv1.weight)
print(net.conv1.bias)

torch.Size([6, 1, 5, 5])
torch.Size([6])
Parameter containing:
tensor([[[[ 0.0975, -0.0157, -0.1039,  0.1130,  0.1273],
          [-0.0135,  0.1003, -0.0613,  0.1766,  0.1645],
          [-0.0663, -0.0308, -0.0011, -0.0568,  0.0375],
          [-0.0745, -0.1873, -0.1515,  0.0814,  0.1239],
          [-0.0157,  0.0898, -0.0053, -0.0913, -0.0619]]],


        [[[-0.1164, -0.0256, -0.1833,  0.0540, -0.0682],
          [-0.1117, -0.1453, -0.0989,  0.0072,  0.1543],
          [-0.0564, -0.0168,  0.0897, -0.0155, -0.1630],
          [ 0.0544,  0.1647, -0.1197, -0.0425, -0.0618],
          [-0.1842, -0.0217, -0.0585, -0.1931, -0.0783]]],


        [[[-0.1385, -0.1656, -0.0811,  0.1474,  0.1322],
          [ 0.1125, -0.0981, -0.0091,  0.1491, -0.0134],
          [-0.0956, -0.1975, -0.1971,  0.0567,  0.1376],
          [ 0.0806,  0.1090,  0.1444, -0.1441, -0.1795],
          [ 0.1359, -0.0525, -0.1245, -0.1098,  0.1019]]],


        [[[ 0.1100, -0.1209, -0.0287, -0.1516, -0.0468],
          [ 0

Il suffit de définir la fonction ``forward`` et la fonction ``backward``
la fonction (où les gradients sont calculés) est automatiquement définie pour vous
en utilisant ``autograd``.
Vous pouvez utiliser n'importe laquelle des opérations Tensor dans la fonction « forward ».

Les paramètres apprenables d'un modèle sont renvoyés par ``net.parameters()``



In [11]:
params = list(net.parameters())
print(len(params))
#print(params[0].size())  # conv1's .weight


for param in params:
  print(param.shape)

nb_params = 0
for param in params:
  nb_params += torch.prod(torch.tensor(param.shape))
print('le nombre de paramètres ajustables est : ' +str(nb_params))

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])
le nombre de paramètres ajustables est : tensor(61706)


**Attention !**
Quand on parle du nombre de paramètres d'un réseau, on parle du nombre de nombres réels dans les matrices, biais et noyaux impliqués dans le réseau (comme dans le cours).

 <font color='blue'> **Question** : Combien de paramètres a ce réseau ? </font>

Essayons d'envoyer une image aléatoire en entrée.


 <font color='blue'> **Question** : Testez le réseau sur une entrée aléatoire qui est une image en niveaux de gris (un canal) de taille $32 \times 32$.</font>

In [13]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0248,  0.1461,  0.0887, -0.0025, -0.0383,  0.1466, -0.1630, -0.0270,
         -0.0570,  0.0461]], grad_fn=<AddmmBackward0>)


##  <font color='blue'> Exercice </font>

1. Nous aimerions définir un réseau qui classe les images de taille $(3,32,32)$.
Construire une classe Net() qui applique :
- une convolution avec $6$ canaux en sortie et un noyau de taille $5 \times 5$, suivi d'un ReLu
- un max pooling de taille $2 \times 2$
- une convolution avec $16$ de canaux en sortie et un noyau de taille $3 \times 3$, suivi d'un ReLu
- un  max pooling de taille $2 \times 2$
- une couche linéaire avec une taille de sortie de $120$, suivie d'un ReLu
- une couche linéaire avec une taille de sortie de $84$, suivie d'un ReLu
- une couche linéaire avec une taille de sortie de $10$

Vérifiez que cela fonctionne avec une entrée aléatoire.

2. Dessinez le réseau comme cela est fait dans les dernières slides du cours.

In [25]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [26]:
x = torch.randn(1,3,32,32)
net(x)

tensor([[-0.0586,  0.0880, -0.0691, -0.1053, -0.0279, -0.0493,  0.0395,  0.0799,
         -0.1162, -0.0666]], grad_fn=<AddmmBackward0>)